# Feedforward neural networks in TensorFlow

Perimeter Institute Computational Tutorial Series

October 29, 2019

Lauren Hayward 

The objective of this tutorial is to become comfortable with using the software library TensorFlow to 
create and train a simple feedforward neural network for supervised learning.

Let us start by generating a random dataset of two-dimensional points with $K$ branches. For each datapoint $\mathbf{x} = (x_1, x_2)$, the label is the branch index such that $y = 0, 1, \ldots K-2$ or $K-1$.
Our goal is to implement a neural network capable of classifying the branches.


In [0]:
import matplotlib.pyplot as plt
import numpy as np

############################################################################
####################### CREATE AND PLOT THE DATA SET #######################
############################################################################

N = 50 # number of points per branch
K = 3  # number of branches

N_train = N*K # total number of points in the training set
x_train = np.zeros((N_train,2)) # matrix containing the 2-dimensional datapoints
y_train = np.zeros(N_train, dtype='uint8') # labels (not in one-hot representation)

mag_noise = 0.3  # controls how much noise gets added to the data
dTheta    = 4    # difference in theta in each branch

### Data generation: ###
for j in range(K):
  ix = range(N*j,N*(j+1))
  r = np.linspace(0.01,1,N) # radius
  t = np.linspace(j*(2*np.pi)/K,j*(2*np.pi)/K + dTheta,N) + np.random.randn(N)*mag_noise # theta
  x_train[ix] = np.c_[r*np.cos(t), r*np.sin(t)]
  y_train[ix] = j

### Plot the data set: ###
fig = plt.figure(1, figsize=(5,5))
plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train, s=40)#, cmap=plt.cm.Spectral)
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.show()

This network will compare its output with labels in the so-called *one-hot encoding*.
For a given label $y=k$, the corresponding one-hot encoding is a $K$-dimensional vector with all entries zero 
except for the $k^\text{th}$ entry (which has value 1).
So, for example, when $K=3$ the one-hot encodings for the labels are
\begin{equation*}
0 \rightarrow \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}, \qquad
1 \rightarrow \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix}, \qquad
2 \rightarrow \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}.
\end{equation*}

**Exercise #1:** Run the code below, which first defines the structure of the neural network 
and then uses the dataset to train this network. 
Look at how this code attempts to classify the two-dimensional space.
You should find that the resulting classifier separates the two-dimensional space using lines, 
and thus does a poor job of representing the data.

In [0]:
%matplotlib inline
from IPython import display
import tensorflow as tf
import time

############################################################################
##################### DEFINE THE NETWORK ARCHITECTURE ######################
############################################################################

### Create placeholders for the input data and labels ###
### (we'll input actual values when we ask TensorFlow to run an actual computation later) ###
x = tf.placeholder(tf.float32, [None, 2]) # input data
y = tf.placeholder(tf.int32,[None])       # labels

### Layer 1: ###
W1 = tf.Variable( tf.random_normal([2, K], mean=0.0, stddev=0.01, dtype=tf.float32) )
b1 = tf.Variable( tf.zeros([K]) )
z1 = tf.matmul(x, W1) + b1
a1 = tf.nn.sigmoid( z1 )

### Network output: ###
aL = a1

### Cost function: ###
### (measures how far off our model is from the labels) ###
y_onehot = tf.one_hot(y,depth=K) # labels are converted to one-hot representation
eps=0.0000000001 # to prevent the logs from diverging
cross_entropy = tf.reduce_mean(-tf.reduce_sum( y_onehot * tf.log(aL+eps) +  (1.0-y_onehot )*tf.log(1.0-aL +eps) , reduction_indices=[1]))
cost_func = cross_entropy

### Use backpropagation to minimize the cost function using the gradient descent algorithm: ###
learning_rate  = 1.0 # hyperparameter
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_func)

N_epochs = 20000 # number of times to run gradient descent

##############################################################################
################################## TRAINING ##################################
##############################################################################
sess = tf.Session()
sess.run(tf.global_variables_initializer())

epoch_list    = []
cost_training = []
acc_training  = []

############ Function for plotting: ############
def updatePlot():

    ### Generate coordinates covering the whole plane: ###
    padding = 0.1
    spacing = 0.02
    x1_min, x1_max = x_train[:, 0].min() - padding, x_train[:, 0].max() + padding
    x2_min, x2_max = x_train[:, 1].min() - padding, x_train[:, 1].max() + padding
    x1_grid, x2_grid = np.meshgrid(np.arange(x1_min, x1_max, spacing),
                         np.arange(x2_min, x2_max, spacing))

    NN_output       = sess.run(aL,feed_dict={x:np.c_[x1_grid.ravel(), x2_grid.ravel()]})
    predicted_class = np.argmax(NN_output, axis=1)

    ### Plot the classifier: ###
    plt.subplot(121)
    plt.contourf(x1_grid, x2_grid, predicted_class.reshape(x1_grid.shape), K, alpha=0.8)
    plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train, s=40)
    plt.xlim(x1_grid.min(), x1_grid.max())
    plt.ylim(x2_grid.min(), x2_grid.max())
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')

    ### Plot the cost function during training: ###
    plt.subplot(222)
    plt.plot(epoch_list,cost_training,'o-')
    plt.xlabel('Epoch')
    plt.ylabel('Training cost')

    ### Plot the training accuracy: ###
    plt.subplot(224)
    plt.plot(epoch_list,acc_training,'o-')
    plt.xlabel('Epoch')
    plt.ylabel('Training accuracy')
############ End of plotting function ############

### Train for several epochs: ###
for epoch in range(N_epochs):
    sess.run(train_step, feed_dict={x: x_train,y:y_train}) #run gradient descent
    
    ### Update the plot and print results every 500 epochs: ###
    if epoch % 500 == 0:
        cost = sess.run(cost_func,feed_dict={x:x_train, y:y_train})
        NN_output = sess.run(aL,feed_dict={x:x_train, y:y_train})
        predicted_class = np.argmax(NN_output, axis=1)
        accuracy = np.mean(predicted_class == y_train)
    
        #print( "Iteration %d:\n  Training cost %f\n  Training accuracy %f\n" % (epoch, cost, accuracy) )
    
        epoch_list.append(epoch)
        cost_training.append(cost)
        acc_training.append(accuracy)
        
        ### Update the plot of the resulting classifier: ###
        fig = plt.figure(2,figsize=(10,5))
        fig.subplots_adjust(hspace=.3,wspace=.3)
        plt.clf()
        updatePlot()
        display.display(plt.gcf())
        display.clear_output(wait=True)
        #time.sleep(0.1) #Uncomment this line if you want to slow down the rate of plot updates

**Exercise #2:** Look through the section of code marked `DEFINE THE NETWORK ARCHITECTURE`.
On paper, draw the neural network corresponding to the one in the code for the case of $K$ branches. 
Pay particular attention to the number of neurons in each layer.


**Exercise #3:** Add in a hidden layer with 4 neurons and study how this hidden layer changes the output. 
On paper, draw the neural network in this case.

**Exercise #4:** Replace the sigmoid activation function on the first layer with a rectified linear unit (ReLU), and study how the 
choice of activation function changes the output.

**Exercise #5:** Change the cost function so that it is computed using the mean-squared error (MSE) instead of the cross-entropy,
and study how the choice of cost function changes the output.

**Exercise #6:** Study the effects of increasing and decreasing the `learning_rate` hyperparameter.
Examine these effects using both the cross-entropy and mean-squared error cost functions.

**Exercise #7:** Explain why the $K$-dimensional one-hot encoding is useful. What do you think would happen if you used a one-dimensional label (such that $y=0,1,\ldots, K-1$ or $K$) instead?

**Exercise #8:** Study how the neural network's accurary changes as a function of:


*   the number of neurons in the hidden layer
*   `mag_noise` (the magnitude of noise in the data)
*   the number of different labels `K`



